In [2]:
import pickle
import os
import concurrent.futures
from tqdm import tqdm
def create_folder(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def dump(path, data):
    with open(path+'.pickle', 'wb') as f:
        pickle.dump(data, f, protocol=2)

def dump_dict(d, base_path, depth, current_depth = 1, base_progress = ""):
    create_folder(base_path)
    i = 0
    futures = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=1000) as executor:
        for key in d:
            progress = base_progress + f"({i} / {len(d)-1})"
            path = os.path.join(base_path, str(key))
            if isinstance(d[key], dict) and current_depth < depth:
                print(progress)
                dump_dict(d[key], path, depth, current_depth+1, progress + " - ")
            else:
                futures.append(executor.submit(dump, path, d[key]))
            i+=1
        concurrent.futures.wait(futures)

In [3]:
import pandas as pd
import os
import re
import pickle

offsets = {}
pattern = re.compile(r'offsets-(ae|se)-(.*).txt')
for root, dirs, files in os.walk("data"):
    for file in files:
        match = pattern.match(file)
        if match:
            gameVersion = match.group(1)
            version = match.group(2)
            df = pd.read_csv("data/"+file, sep='\t', header=None)
            df[0] = df[0].apply(lambda x: int(x))
            df[1] = df[1].apply(lambda x: "0x"+x)

            offsets[version]={
                "game-version":gameVersion,
                "skyrim-to-address": dict(zip(df[1], df[0])),
                "address-to-skyrim": dict(zip(df[0], df[1])),
            }


dump_dict(offsets,".\\data\\offsets\\", 3)


# with open('data/offsets.pickle', 'wb')  as f:
#     pickle.dump(offsets, f, protocol=2)

(0 / 1)
(0 / 1) - (1 / 2)
(0 / 1) - (2 / 2)
